<a href="https://colab.research.google.com/github/hdpark1208/StudyCode/blob/main/6)%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./auto-mpg.csv',header=None)

FileNotFoundError: ignored

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 

In [ ]:
df.head()

In [ ]:
pd.set_option('display.max_columns',10)
df.head()

In [ ]:
df['horsepower'].unique()

In [ ]:
df['horsepower'].replace('?',np.nan,inplace=True)

In [ ]:
df.dropna(subset=['horsepower'],axis=0,inplace=True)
df['horsepower']=df['horsepower'].astype('float')
df.describe()

In [ ]:
ndf = df[['mpg','cylinders','horsepower','weight']]
ndf.head()

In [ ]:
ndf.plot(kind='scatter',x='weight',y='mpg',c='blue',s=10,figsize=(10,6))
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
sns.regplot(x='weight',y='mpg',data=ndf,ax=ax1)
sns.regplot(x='weight',y='mpg',data=ndf,ax=ax2,fit_reg=False)
plt.show()



In [ ]:
X = ndf[['weight']]
y = ndf['mpg']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=10)


In [ ]:
print(len(X_train),len(X_test))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
r_square = lr.score(X_test,y_test)
r_square

In [ ]:
y_hat = lr.predict(X)

plt.figure(figsize=(10,5))
ax1= sns.distplot(y,hist=False,label='y')
ax2= sns.distplot(y_hat,hist=False,label='y_hat',ax=ax1)
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
print(X_train.shape)
X_train_poly.shape

In [ ]:
plr = LinearRegression()
plr.fit(X_train_poly,y_train)

In [ ]:
X_test_poly = poly.fit_transform(X_test)
r_square = plr.score(X_test_poly,y_test)
r_square

In [ ]:
y_hat_test = plr.predict(X_test_poly)

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
ax.plot(X_train,y_train,'o',label='Train')
ax.plot(X_test,y_hat_test,'r+',label='Predict')
ax.legend(loc='best')
plt.xlabel('weight')
plt.ylabel('mpg')
plt.show()

In [3]:
import pandas as pd
import seaborn as sns

In [4]:
df= sns.load_dataset('titanic')

In [5]:
pd.set_option('display.max_columns',15)

In [6]:
rdf = df.drop(['deck','embark_town'],axis=1)
rdf = rdf.dropna(subset=['age'],how='any',axis=0)

In [8]:
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq,inplace=True)

In [9]:
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]

In [10]:
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,onehot_sex],axis=1)

In [11]:
onehot_embarked = pd.get_dummies(ndf['embarked'],prefix='town')
ndf = pd.concat([ndf,onehot_embarked],axis=1)

ndf.drop(['sex','embarked'],axis=1,inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [12]:
X=ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']]
y=ndf['survived']

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=10)

print(X_train.shape,X_test.shape)

(499, 9) (215, 9)


In [14]:
from sklearn import svm

In [15]:
svm_model = svm.SVC(kernel='rbf')
svm_model.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
y_hat = svm_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 0 0 0 1 0 0 0 0]
[0 0 1 0 0 1 1 1 0 0]


In [18]:
from sklearn import metrics
svm_matrix = metrics.confusion_matrix(y_test,y_hat)

print(svm_matrix)


[[118   7]
 [ 79  11]]


In [23]:
svm_report = metrics.classification_report(y_test,y_hat)
svm_report

'              precision    recall  f1-score   support\n\n           0       0.60      0.94      0.73       125\n           1       0.61      0.12      0.20        90\n\n    accuracy                           0.60       215\n   macro avg       0.61      0.53      0.47       215\nweighted avg       0.60      0.60      0.51       215\n'

In [24]:
type(svm_report)

str